In [3]:
import os
import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import box, Polygon
import glob

In [4]:
or_quads = gpd.read_file("../data/interim/oregon_quarter_quads.shp")
big_bbox = or_quads.geometry.bounds.agg(
    {"minx": "min", "miny": "min", "maxx": "max", "maxy": "max"}
).values

In [5]:
def get_parcel_url(bbox, inSR, **kwargs):
    BASE_URL = "".join(
        [
            "https://utility.arcgis.com/usrsvcs/servers/",
            "78bbb0d0d9c64583ad5371729c496dcc/rest/services/",
            "Secure/DOR_ORMAP/MapServer/3/query?",
        ]
    )

    params = dict(
        f="geojson",
        returnGeometry="true",
        spatialRel="esriSpatialRelIntersects",
        geometry=(
            f'{{"xmin":{bbox[0]},"ymin":{bbox[1]},'
            f'"xmax":{bbox[2]},"ymax":{bbox[3]},'
            f'"spatialReference":{{"wkid":{inSR}}}}}'
        ),
        geometryType="esriGeometryEnvelope",
        outFields="*",
        outSR=inSR,
    )
    for key, value in kwargs.items():
        params.update({key: value})

    url = requests.Request("GET", BASE_URL, params=params).prepare().url
    return url


def get_parcel_ids(bbox, inSR):
    url = get_parcel_url(bbox, inSR, returnIdsOnly="true")
    try:
        r = requests.get(url)
        ids = r.json()["objectIds"]
        return ids
    except:
        return r

In [27]:
def parcel_post(bbox, inSR, min_id, max_id, **kwargs):
    BASE_URL = "".join(
        [
            "https://utility.arcgis.com/usrsvcs/servers/",
            "78bbb0d0d9c64583ad5371729c496dcc/rest/services/",
            "Secure/DOR_ORMAP/MapServer/3/query?",
        ]
    )

    params = dict(
        f="geojson",
        where=f"(OBJECTID >= {min_id} AND OBJECTID <= {max_id})",
        # returnGeometry='true',
        # spatialRel='esriSpatialRelIntersects',
        # geometry=(f'{{"xmin":{bbox[0]},"ymin":{bbox[1]},'
        #           f'"xmax":{bbox[2]},"ymax":{bbox[3]},'
        #           f'"spatialReference":{{"wkid":{inSR}}}}}'),
        # geometryType='esriGeometryEnvelope',
        outFields="*",
        outSR=inSR,
    )
    for key, value in kwargs.items():
        params.update({key: value})

    r = requests.post(BASE_URL, data=params, timeout=60.0)
    return r

In [7]:
all_ids = get_parcel_ids(big_bbox, inSR=2992)

In [23]:
def get_parcels(min_id, max_id, epsg=2992):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"
    }
    url = "https://utility.arcgis.com/usrsvcs/servers/78bbb0d0d9c64583ad5371729c496dcc/rest/services/Secure/DOR_ORMAP/MapServer/3/query?f=geojson&returnGeometry=true&outFields=*&outSR={}&where=(OBJECTID >= {} AND OBJECTID <= {})".format(
        epsg, min_id, max_id
    )
    r = requests.get(url, headers=headers)

    if r.status_code != 200:
        print("Response [{}]: {}".format(r.status_code, r.reason))
    try:
        gdf = gpd.GeoDataFrame.from_features(r.json(), crs=epsg)
    except:
        gdf = None
    return r, gdf

In [9]:
j = 0
for i in range(0, len(all_ids), 1000):
    start_id, end_id = i, i + 1000
    ids = all_ids[start_id:end_id]
    outfile = "../data/interim/oregon_parcels/{:07d}_{:07d}_parcels.geojson".format(
        start_id, end_id
    )
    if not os.path.exists(outfile):
        try:
            r, gdf = get_parcels(start_id, end_id, 2992)
            gdf.to_file(outfile, driver="GeoJSON")
        except:
            basename = os.path.basename(outfile)
            print(f"Failed on {basename}.")

    ## report progress
    if (j + 1) % 10 == 0:
        print(j + 1, end="")
    else:
        print(".", end="")
    if (j + 1) % 100 == 0:
        print()

    j += 1

.........10.........20.........30.........40.........50.........60.........70.........80.........90.........100
.........110.........120.........130.........140.........150.........160.........170.........180.........190.........200
.........210.........220.........230.........240.........250.........260.........270.........280.........290.........300
.........310.........320.........330.........340.........350.........360.........370.........380.........390.........400
.........410.........420.........430.........440.........450.........460.........470.........480.........490.........500
.........510.........520....Failed on 0524000_0525000_parcels.geojson.
.....530.........540.........550.........560.........570.........580.........590.........600
.........610.........620.........630.........640.........650.........660.........670.........680.........690.........700
.........710.........720.........730.........740.........750.........760.........770.........780.........790.........80

In [40]:
# NOT_DOWNLOADED = [524000, 524001, 524002, 524004, 524005, 524006, 524007, 524008, 524009, 524017, 524018, 524026, 524027, 524028, 524041, 524049, 524050, 524051, 524052, 524053, 524054, 524058, 524059, 524060, 524066, 524067, 524068, 524069, 524073, 524074, 524078, 524084, 524086, 524087, 524089, 524090, 524091, 524095, 524097, 524098, 524099, 524102, 524108, 524110, 524111, 524114, 524115, 524118, 524119, 524123, 524124, 524125, 524126, 524127, 524128, 524130, 524133, 524145, 524152, 524162, 524163, 524182, 524185, 524197, 524198, 524199, 524204, 524205, 524206, 524207, 524209, 524210, 524213, 524214, 524215, 524216, 524243, 524244, 524245, 524247, 524248, 524249, 524257, 524268, 524269, 524270, 524273, 524279, 524280, 524281, 524282, 524283, 524284, 524285, 524286, 524287, 524288, 524289, 524290, 524291, 524292, 524293, 524295, 524296, 524297, 524302, 524303, 524306, 524307, 524308, 524312, 524314, 524315, 524316, 524327, 524331, 524334, 524335, 524336, 524366, 524386, 524391, 524392, 524394, 524395, 524396, 524397, 524398, 524399, 524403, 524405, 524406, 524407, 524408, 524409, 524410, 524411, 524412, 524413, 524415, 524416, 524417, 524419, 524420, 524421, 524422, 524423, 524424, 524425, 524426, 524427, 524428, 524429, 524430, 524433, 524434, 524435, 524436, 524437, 524438, 524439, 524440, 524444, 524445, 524446, 524448, 524450, 524451, 524455, 524456, 524457, 524458, 524459, 524460, 524461, 524462, 524463, 524464, 524465, 524466, 524467, 524468, 524469, 524470, 524471, 524472, 524473, 524474, 524479, 524480, 524484, 524485, 524486, 524487, 524488, 524491, 524492, 524493, 524494, 524495, 524496, 524497, 524498, 524499, 524500, 524501, 524502, 524503, 524507, 524508, 524509, 524510, 524511, 524512, 524513, 524514, 524515, 524516, 524517, 524518, 524519, 524520, 524522, 524531, 524532, 524533, 524534, 524545, 524546, 524547, 524548, 524549, 524550, 524551, 524552, 524553, 524554, 524555, 524556, 524568, 524569, 524570, 524571, 524572, 524575, 524576, 524578, 524579, 524580, 524581, 524585, 524586, 524591, 524592, 524593, 524594, 524604, 524606, 524607, 524608, 524609, 524610, 524611, 524613, 524614, 524615, 524616, 524620, 524622, 524623, 524624, 524625, 524626, 524634, 524635, 524636, 524642, 524643, 524644, 524645, 524647, 524648, 524650, 524651, 524652, 524653, 524654, 524655, 524656, 524657, 524658, 524659, 524660, 524661, 524663, 524664, 524665, 524666, 524668, 524674, 524677, 524678, 524679, 524680, 524685, 524686, 524687, 524688, 524689, 524690, 524693, 524694, 524697, 524698, 524706, 524707, 524708, 524709, 524710, 524711, 524712, 524714, 524715, 524716, 524717, 524718, 524719, 524720, 524721, 524722, 524723, 524724, 524725, 524726, 524727, 524728, 524729, 524730, 524731, 524732, 524733, 524734, 524744, 524745, 524746, 524747, 524748, 524749, 524750, 524751, 524753, 524754, 524755, 524757, 524758, 524759, 524776, 524781, 524782, 524783, 524784, 524785, 524786, 524787, 524788, 524789, 524791, 524794, 524799, 524806, 524807, 524808, 524809, 524810, 524811, 524812, 524813, 524814, 524817, 524819, 524820, 524821, 524822, 524823, 524825, 524826, 524827, 524829, 524835, 524836, 524837, 524840, 524841, 524842, 524843, 524844, 524845, 524847, 524848, 524849, 524850, 524851, 524852, 524853, 524854, 524855, 524856, 524857, 524858, 524859, 524860, 524864, 524868, 524869, 524870, 524871, 524872, 524873, 524881, 524894, 524895, 524902, 524906, 524921, 524922, 524923, 524924, 524925, 524926, 524927, 524928, 524929, 524930, 524931, 524932, 524938, 524939, 524940, 524951, 524952, 524953, 524954, 524955, 524959, 524960, 524961, 524962, 524973, 524979, 524980, 524981, 524982, 524983, 524984, 524985, 524987, 524989, 524990, 524991, 524992, 524993, 524994, 524995]
# BASE_URL = ''.join([
#         'https://utility.arcgis.com/usrsvcs/servers/',
#         '78bbb0d0d9c64583ad5371729c496dcc/rest/services/',
#         'Secure/DOR_ORMAP/MapServer/3/query?',
#     ])

# params = dict(f='geojson',
#               objectIds=','.join([str(x) for x in NOT_DOWNLOADED]),
#               # returnGeometry='true',
#               # spatialRel='esriSpatialRelIntersects',
#               # geometry=(f'{{"xmin":{bbox[0]},"ymin":{bbox[1]},'
#               #           f'"xmax":{bbox[2]},"ymax":{bbox[3]},'
#               #           f'"spatialReference":{{"wkid":{inSR}}}}}'),
#               # geometryType='esriGeometryEnvelope',
#               outFields='*',
#               outSR=2992)

# r = requests.post(BASE_URL, data=params, timeout=60.0)